# Catboost

In [ ]:
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np

def preprocess_for_catboost(input_train_df, input_test_df, target_column='isFraud'):
    # Work on local copies to avoid modifying global variables
    train_df = input_train_df.copy()
    test_df = input_test_df.copy()

    # Drop target column
    if target_column in train_df.columns:
        train_df_labels = train_df[target_column]  # Save target
        train_df = train_df.drop(columns=[target_column])
    else:
        train_df_labels = None

    # Identify numeric and categorical columns
    numeric_columns = train_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_columns = train_df.select_dtypes(include=['object']).columns.tolist()

    # Ensure columns exist in both train and test
    numeric_columns = [col for col in numeric_columns if col in test_df.columns]
    categorical_columns = [col for col in categorical_columns if col in test_df.columns]

    # Handle missing values
    numeric_imputer = SimpleImputer(strategy='median')
    train_df[numeric_columns] = numeric_imputer.fit_transform(train_df[numeric_columns])
    test_df[numeric_columns] = numeric_imputer.transform(test_df[numeric_columns])

    categorical_imputer = SimpleImputer(strategy='constant', fill_value='missing')
    train_df[categorical_columns] = categorical_imputer.fit_transform(train_df[categorical_columns])
    test_df[categorical_columns] = categorical_imputer.transform(test_df[categorical_columns])

    # Combine numeric and categorical columns
    all_columns = numeric_columns + categorical_columns
    train_df = train_df[all_columns]
    test_df = test_df[all_columns]

    # Return processed data and categorical feature indices
    categorical_feature_indices = [all_columns.index(col) for col in categorical_columns]
    return train_df, test_df, train_df_labels, categorical_feature_indices


In [ ]:
filled_train, filled_test, train_y, cat_features = preprocess_for_catboost(train_processed, test_processed)

print("Preprocessed Train Shape:", filled_train.shape)
print("Preprocessed Test Shape:", filled_test.shape)
print("Categorical Features:", cat_features)


Preprocessed Train Shape: (590540, 27)
Preprocessed Test Shape: (506691, 27)
Categorical Features: [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]


In [ ]:
train_y = train_processed['isFraud']

In [ ]:
train_y.value_counts()

,count
isFraud,
0,569877
1,20663


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.6 MB/s eta 0:00:00


In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 14.5 MB/s eta 0:00:00


In [ ]:
import optuna
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
y_train = train_y

In [ ]:
import optuna
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# Target variable
y_train = train_y

# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters for CatBoost
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "depth": trial.suggest_int("depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.5, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 10.0, log=True),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.0, 10.0),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "loss_function": "Logloss",
        "eval_metric": "AUC",
        "task_type": "GPU",  # Change to "GPU" if running on GPU
        "verbose": False,
        "random_state": 42,
    }

    # Cross-validation setup
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    auc_scores = []

    for train_idx, val_idx in cv.split(filled_train, y_train):
        X_train, X_val = filled_train.iloc[train_idx], filled_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

        # Create CatBoost Pool
        train_pool = Pool(X_train, y_train_fold, cat_features=cat_features)
        val_pool = Pool(X_val, y_val_fold, cat_features=cat_features)

        # Train CatBoost model
        model = CatBoostClassifier(**params)
        model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=50, verbose=False)

        # Predict probabilities for validation set
        y_val_pred_proba = model.predict_proba(X_val)[:, 1]  # Probabilities for class 1

        # Calculate AUC
        auc = roc_auc_score(y_val_fold, y_val_pred_proba)
        auc_scores.append(auc)
        print(np.mean(auc_scores))

    # Return the mean AUC (maximize this)
    return np.mean(auc_scores)

# Run Optuna optimization
study = optuna.create_study(direction="maximize", study_name="CatBoost AUC Tuning")
study.optimize(objective, n_trials=50)

# Print best parameters
print("Best parameters:", study.best_trial.params)



[I 2024-12-08 03:55:40,504] A new study created in memory with name: CatBoost AUC Tuning
Default metric period is 5 because AUC is/are not implemented for GPU


0.9043881077084825


Default metric period is 5 because AUC is/are not implemented for GPU


0.9058401088677082


Default metric period is 5 because AUC is/are not implemented for GPU


...

Default metric period is 5 because AUC is/are not implemented for GPU


0.967955032533131


Default metric period is 5 because AUC is/are not implemented for GPU


0.967695418873021


Default metric period is 5 because AUC is/are not implemented for GPU


In [ ]:
best_params = {'iterations': 709, 'depth': 5, 'learning_rate': 0.22378524791213256, 'l2_leaf_reg': 0.030725954622609027, 'bagging_temperature': 0.035983637250438405, 'random_strength': 5.650565084985939, 'border_count': 89}

In [ ]:
best_params_2 = {'iterations': 749, 'depth': 9, 'learning_rate': 0.23122004584043554, 'l2_leaf_reg': 1.4636245026812997, 'bagging_temperature': 0.17560018444882333, 'random_strength': 4.758805450016254, 'border_count': 222}

In [ ]:
best_params

{'iterations': 709,
 'depth': 5,
 'learning_rate': 0.22378524791213256,
 'l2_leaf_reg': 0.030725954622609027,
 'bagging_temperature': 0.035983637250438405,
 'random_strength': 5.650565084985939,
 'border_count': 89}

In [ ]:
best_params_2

{'iterations': 749,
 'depth': 9,
 'learning_rate': 0.23122004584043554,
 'l2_leaf_reg': 1.4636245026812997,
 'bagging_temperature': 0.17560018444882333,
 'random_strength': 4.758805450016254,
 'border_count': 222}

In [ ]:
# Train the final model with the best parameters on the full dataset
best_params = study.best_trial.params
final_pool = Pool(filled_train, y_train, cat_features=cat_features)
final_model = CatBoostClassifier(**best_params, verbose=100)
final_model.fit(final_pool)

# Make predictions on the test set
test_pool = Pool(filled_test, cat_features=cat_features)
test_pred_proba = final_model.predict_proba(test_pool)[:, 1]

# Create submission
submission = pd.DataFrame({
    "TransactionID": test_processed["TransactionID"],
    "isFraud": test_pred_proba
})
submission.to_csv("catboost_auc_submission.csv", index=False)

print("Submission file created: catboost_auc_submission.csv")

0:	learn: 0.4076108	total: 300ms	remaining: 3m 32s
100:	learn: 0.0733359	total: 57s	remaining: 5m 43s
200:	learn: 0.0659926	total: 1m 51s	remaining: 4m 42s
300:	learn: 0.0616673	total: 2m 46s	remaining: 3m 45s
400:	learn: 0.0590472	total: 3m 40s	remaining: 2m 49s
500:	learn: 0.0571702	total: 4m 34s	remaining: 1m 54s
600:	learn: 0.0555282	total: 5m 29s	remaining: 59.2s
700:	learn: 0.0543717	total: 6m 21s	remaining: 4.36s
708:	learn: 0.0542038	total: 6m 26s	remaining: 0us
Submission file created: catboost_auc_submission.csv


In [ ]:
# Train the final model with the best parameters on the full dataset
#best_params = study.best_trial.params
final_pool = Pool(filled_train, y_train, cat_features=cat_features)
final_model = CatBoostClassifier(**best_params_2, verbose=100)
final_model.fit(final_pool)

# Make predictions on the test set
test_pool = Pool(filled_test, cat_features=cat_features)
test_pred_proba = final_model.predict_proba(test_pool)[:, 1]

# Create submission
submission = pd.DataFrame({
    "TransactionID": test_processed["TransactionID"],
    "isFraud": test_pred_proba
})
submission.to_csv("catboost_auc_submission_fine_tuned_with_more_trials.csv", index=False)

print("Submission file created: catboost_auc_submission.csv")

0:	learn: 0.4075424	total: 503ms	remaining: 6m 16s
100:	learn: 0.0678686	total: 58.7s	remaining: 6m 16s
200:	learn: 0.0574476	total: 2m 5s	remaining: 5m 41s
300:	learn: 0.0524879	total: 3m 10s	remaining: 4m 43s
400:	learn: 0.0481044	total: 4m 15s	remaining: 3m 41s
500:	learn: 0.0444347	total: 5m 19s	remaining: 2m 38s
600:	learn: 0.0412870	total: 6m 24s	remaining: 1m 34s
700:	learn: 0.0387877	total: 7m 27s	remaining: 30.7s
748:	learn: 0.0375502	total: 7m 58s	remaining: 0us
Submission file created: catboost_auc_submission.csv


## Catboost with LOG loss evaluation with 30 trials

In [ ]:
import optuna
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

# Target variable
y_train = train_y

# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters for CatBoost
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "depth": trial.suggest_int("depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.5, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 10.0, log=True),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.0, 10.0),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "loss_function": "Logloss",
        "eval_metric": "Logloss",
        "task_type": "GPU",  # Change to "GPU" if running on GPU
        "verbose": False,
        "random_state": 42,
    }

    # Cross-validation setup
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    log_losses = []

    for train_idx, val_idx in cv.split(filled_train, y_train):
        X_train, X_val = filled_train.iloc[train_idx], filled_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

        # Create CatBoost Pool
        train_pool = Pool(X_train, y_train_fold, cat_features=cat_features)
        val_pool = Pool(X_val, y_val_fold, cat_features=cat_features)

        # Train CatBoost model
        model = CatBoostClassifier(**params)
        model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=50, verbose=False)

        # Predict probabilities for validation set
        y_val_pred_proba = model.predict_proba(X_val)[:, 1]  # Probabilities for class 1

        # Calculate Log Loss
        log_losses.append(log_loss(y_val_fold, y_val_pred_proba + 1e-15))  # Add small epsilon to avoid log(0)

    # Return the mean log loss (minimize this)
    print(np.mean(log_losses))
    return np.mean(log_losses)

# Run Optuna optimization
study = optuna.create_study(direction="minimize", study_name="CatBoost Log Loss Tuning")
study.optimize(objective, n_trials=50)

# Print best parameters
print("Best parameters:", study.best_trial.params)

# # Train the final model with the best parameters on the full dataset
# best_params = study.best_trial.params
# final_pool = Pool(filled_train, y_train, cat_features=cat_features)
# final_model = CatBoostClassifier(**best_params, verbose=100)
# final_model.fit(final_pool)

# # Make predictions on the test set
# test_pool = Pool(filled_test, cat_features=cat_features)
# test_pred_proba = final_model.predict_proba(test_pool)[:, 1]

# # Create submission
# submission = pd.DataFrame({
#     "TransactionID": test_processed["TransactionID"],
#     "isFraud": test_pred_proba
# })
# submission.to_csv("catboost_logloss_submission.csv", index=False)

# print("Submission file created: catboost_logloss_submission.csv")


[I 2024-12-07 20:03:38,297] A new study created in memory with name: CatBoost Log Loss Tuning
[I 2024-12-07 20:08:07,831] Trial 0 finished with value: 0.051666531659739265 and parameters: {'iterations': 628, 'depth': 8, 'learning_rate': 0.1568043883162735, 'l2_leaf_reg': 0.04351111154361797, 'bagging_temperature': 0.881323716639531, 'random_strength': 3.738382614647681, 'border_count': 242}. Best is trial 0 with value: 0.051666531659739265.


0.051666531659739265


[I 2024-12-07 20:13:54,880] Trial 1 finished with value: 0.052524361473352335 and parameters: {'iterations': 998, 'depth': 7, 'learning_rate': 0.08744799447668071, 'l2_leaf_reg': 0.0019929958328108823, 'bagging_temperature': 0.7662886573259491, 'random_strength': 7.963089144666048, 'border_count': 158}. Best is trial 0 with value: 0.051666531659739265.


0.052524361473352335


[I 2024-12-07 20:16:07,358] Trial 2 finished with value: 0.06122054973712495 and parameters: {'iterations': 309, 'depth': 7, 'learning_rate': 0.11192074071754711, 'l2_leaf_reg': 2.5630813399738623, 'bagging_temperature': 0.9716364945840094, 'random_strength': 7.537579028624313, 'border_count': 186}. Best is trial 0 with value: 0.051666531659739265.


...

0.04605015597593375


[I 2024-12-07 23:50:55,220] Trial 48 finished with value: 0.04777611948487367 and parameters: {'iterations': 424, 'depth': 10, 'learning_rate': 0.23244405902922669, 'l2_leaf_reg': 0.44578318384961274, 'bagging_temperature': 0.08700153887428552, 'random_strength': 2.1805523087076693, 'border_count': 182}. Best is trial 47 with value: 0.04605015597593375.


0.04777611948487367


[I 2024-12-07 23:56:37,520] Trial 49 finished with value: 0.08101055993373722 and parameters: {'iterations': 698, 'depth': 9, 'learning_rate': 0.0050862600483667095, 'l2_leaf_reg': 1.1610784836429426, 'bagging_temperature': 0.0017270943499896898, 'random_strength': 0.0478708931412557, 'border_count': 200}. Best is trial 47 with value: 0.04605015597593375.


0.08101055993373722
Best parameters: {'iterations': 851, 'depth': 9, 'learning_rate': 0.2297453294159207, 'l2_leaf_reg': 0.4045230071405023, 'bagging_temperature': 0.004527543389180377, 'random_strength': 2.3179713243727047, 'border_count': 184}


In [ ]:
{'iterations': 851, 'depth': 9, 'learning_rate': 0.2297453294159207, 'l2_leaf_reg': 0.4045230071405023, 'bagging_temperature': 0.004527543389180377, 'random_strength': 2.3179713243727047, 'border_count': 184}

In [ ]:
# Train the final model with the best parameters on the full dataset
best_params = study.best_trial.params
final_pool = Pool(filled_train, y_train, cat_features=cat_features)
final_model = CatBoostClassifier(**best_params, verbose=100)
final_model.fit(final_pool)

# Make predictions on the test set
test_pool = Pool(filled_test, cat_features=cat_features)
test_pred_proba = final_model.predict_proba(test_pool)[:, 1]

# Create submission
submission = pd.DataFrame({
    "TransactionID": test_processed["TransactionID"],
    "isFraud": test_pred_proba
})
submission.to_csv("catboost_logloss_submission.csv", index=False)

print("Submission file created: catboost_logloss_submission.csv")

0:	learn: 0.4018979	total: 235ms	remaining: 3m 19s
100:	learn: 0.0602193	total: 1m 48s	remaining: 13m 23s
200:	learn: 0.0501391	total: 3m 35s	remaining: 11m 37s
300:	learn: 0.0444150	total: 5m 20s	remaining: 9m 46s
400:	learn: 0.0404429	total: 7m 6s	remaining: 7m 58s
500:	learn: 0.0373053	total: 8m 51s	remaining: 6m 11s
600:	learn: 0.0343420	total: 10m 36s	remaining: 4m 24s
700:	learn: 0.0318427	total: 12m 19s	remaining: 2m 38s
800:	learn: 0.0295467	total: 14m 5s	remaining: 52.8s
850:	learn: 0.0284451	total: 15m	remaining: 0us
Submission file created: catboost_logloss_submission.csv
